## Runnables 구조 검토

In [1]:
# !pip install -qU langchain langchain-openai faiss-cpu tiktoken

In [2]:
# !pip install -qU grandalf

In [4]:
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    # 텍스트 데이터로부터 FAISS 벡터 저장소를 생성합니다.
    ["Teddy is an AI engineer who loves programming!"],
    embedding=OpenAIEmbeddings(),
)

# 벡터 저장소를 기반으로 retriever를 생성합니다.
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}  

Question: {question}"""

prompt = ChatPromptTemplate.from_template(
    template
)  # 템플릿을 기반으로 ChatPromptTemplate을 생성합니다.

model = ChatOpenAI()  # ChatOpenAI 모델을 초기화합니다.

# chain 을 생성합니다.
chain = (
    # 검색 컨텍스트와 질문을 지정합니다.
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt  # 프롬프트를 생성합니다.
    | model  # 언어 모델을 실행합니다.
    | StrOutputParser()  # 출력 결과를 문자열로 파싱합니다.
)

### 그래프 구성 확인

chain.get_graph() 메서드는 체인의 실행 그래프를 반환합니다.

- 이 메서드는 체인의 각 노드와 노드 간의 연결을 나타내는 그래프 객체를 반환합니다.
- 그래프의 노드는 체인의 각 단계를 나타내며, 에지(edge)는 단계 간의 데이터 흐름을 나타냅니다.

In [6]:
# 체인의 그래프에서 노드를 가져옵니다.
chain.get_graph().nodes

{'697115a949f848319851f8c238a247e8': Node(id='697115a949f848319851f8c238a247e8', name='Parallel<context,question>Input', data=<class 'pydantic.v1.main.RunnableParallel<context,question>Input'>, metadata=None),
 '01e4274e37f947fc912c8aff8c2dcb23': Node(id='01e4274e37f947fc912c8aff8c2dcb23', name='Parallel<context,question>Output', data=<class 'pydantic.v1.main.RunnableParallel<context,question>Output'>, metadata=None),
 '9755830b34d5432c9e030d821e9cb28d': Node(id='9755830b34d5432c9e030d821e9cb28d', name='VectorStoreRetriever', data=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000024417B61E70>), metadata=None),
 'd638c2cc0a5c4091af77f71756a8402a': Node(id='d638c2cc0a5c4091af77f71756a8402a', name='Passthrough', data=RunnablePassthrough(), metadata=None),
 'f7f20490b72f4f53bceae67da6590bcd': Node(id='f7f20490b72f4f53bceae67da6590bcd', name='ChatPromptTemplate', data=ChatPromptTemplate(input_variables=['conte

In [7]:
# 체인의 그래프에서 엣지를 가져옵니다.
chain.get_graph().edges

[Edge(source='3ba7ee11b55642eea7c3cbc3081395ff', target='95289e88cf34467db1b877dc466d1b99', data=None, conditional=False),
 Edge(source='95289e88cf34467db1b877dc466d1b99', target='a53c585d3b37450c97277d6ddec6a2bb', data=None, conditional=False),
 Edge(source='3ba7ee11b55642eea7c3cbc3081395ff', target='6ccaee407e97468da64353f105d6ce88', data=None, conditional=False),
 Edge(source='6ccaee407e97468da64353f105d6ce88', target='a53c585d3b37450c97277d6ddec6a2bb', data=None, conditional=False),
 Edge(source='a53c585d3b37450c97277d6ddec6a2bb', target='1ea5230656dd47e98e05314011f45d06', data=None, conditional=False),
 Edge(source='1ea5230656dd47e98e05314011f45d06', target='9afe7fcc0cf44648ad16f865046a618b', data=None, conditional=False),
 Edge(source='68468d967f3c46cc86a91ebf57b65ccf', target='eea817947a3344d18eab2aa52ce1773b', data=None, conditional=False),
 Edge(source='9afe7fcc0cf44648ad16f865046a618b', target='68468d967f3c46cc86a91ebf57b65ccf', data=None, conditional=False)]

### 그래프 출력

In [8]:
# 체인의 그래프를 ASCII 형식으로 출력합니다.
chain.get_graph().print_ascii()

           +---------------------------------+         
           | Parallel<context,question>Input |         
           +---------------------------------+         
                    **               **                
                 ***                   ***             
               **                         **           
+----------------------+              +-------------+  
| VectorStoreRetriever |              | Passthrough |  
+----------------------+              +-------------+  
                    **               **                
                      ***         ***                  
                         **     **                     
           +----------------------------------+        
           | Parallel<context,question>Output |        
           +----------------------------------+        
                             *                         
                             *                         
                             *                  

### 프롬프트 가져오기

In [9]:
chain.get_prompts()  # 체인에서 사용되는 프롬프트를 가져옵니다.

[ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='Answer the question based only on the following context:\n{context}  \n\nQuestion: {question}'))])]